In [ ]:
!git clone https://github.com/kiyoungkim1/LMkor
!pip3 install -q transformers

from LMkor.examples.gpt3_generation import Inference

fatal: destination path 'LMkor' already exists and is not an empty directory.


In [ ]:
import transformers
from transformers import BertTokenizerFast, TFGPT2LMHeadModel, GPT2LMHeadModel
transformers.logging.set_verbosity_error()

class Inference():
    def __init__(self, model_name, tf_pt='tf'):
        self.tf_pt = tf_pt

        self.tokenizer = BertTokenizerFast.from_pretrained(model_name)
        if self.tf_pt == 'tf':
            self.model = TFGPT2LMHeadModel.from_pretrained(model_name, pad_token_id=self.tokenizer.eos_token_id)
        else:
            self.model = GPT2LMHeadModel.from_pretrained(model_name, pad_token_id=self.tokenizer.eos_token_id)

    def __call__(self, text, howmany=3):
        input_ids = self.tokenizer.encode(text, return_tensors='tf' if self.tf_pt=='tf' else 'pt')
        input_ids = input_ids[:, 1:]  # remove cls token

        outputs = self.model.generate(
            input_ids,
            min_length=30,
            max_length=64,
            do_sample=True,
            top_k=10,
            top_p=0.95,
            no_repeat_ngram_size=2,
            num_return_sequences=howmany
        )
        lists = []
        for idx, generated in enumerate([self.tokenizer.decode(sentence, skip_special_tokens=True) for sentence in outputs]):
            lists.append(generated[len(text):])
        return lists[0]
  
inference = Inference('kykim/gpt3-kor-small_based_on_gpt2')

In [ ]:
import pandas as pd
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/train_data.csv')

## TEXT_GPT

In [ ]:
df_it = df[df['topic_idx'] == 0]
df_it.title = df_it.loc[:, "title"].apply(word_eda)
df_it.reset_index(inplace=True)
df_it = df_it[2170:]

/usr/local/lib/python3.7/dist-packages/pandas/core/generic.py:5170: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [ ]:
from tqdm import tqdm
df_gpt = pd.DataFrame(columns=['title'])
for i in tqdm(df_it.title) :
  df_gpt = df_gpt.append({'title' : inference(i, howmany=1)}, ignore_index=True)

100%|██████████| 2654/2654 [3:30:02<00:00,  4.75s/it]


In [ ]:
df_gpt.to_csv('/content/drive/MyDrive/gpt_it.csv')